In [ ]:
import numpy as np
import re
from keras.models import Sequential
from keras.models import Model
from keras.layers import LSTM, Dense, Dropout, Input, concatenate, BatchNormalization, Activation
from keras.layers.embeddings import Embedding
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from pickle import dump
from pickle import load 
from keras.models import load_model
import pickle
from keras.utils import plot_model

In [ ]:
# Note: Colab pipeline meant for continuing model training 
# We load the processed character text instead of the ready-to-run X and y data
# because uploading X and y data usually too large to upload. 
# We repeat the preprocessing steps 
with open("process_shakespeare.txt", "rb") as fp: 
    char_text = pickle.load(fp)

In [ ]:
length = 40 + 1
sequences = []
for i in range(length, len(char_text)): 
    seq = char_text[i-length:i]
    seq_string = ''.join(seq)
    sequences.append(seq_string)
print(len(sequences))

In [ ]:
num_char = []
for line in sequences: 
    encoded = [mapping[char] for char in line]
    num_char.append(encoded)
print(len(num_char[0]))

In [ ]:
num_char = np.array(num_char)
X, y = num_char[:, :-1], num_char[:, -1]
print(X.shape)
print(y.shape)

In [ ]:
X = np.array([to_categorical(x, num_classes=len(mapping)) for x in X])
y = to_categorical(y, num_classes=len(mapping))
print(X.shape)
print(y.shape)

In [ ]:
# For running locally, you can load directly and then stack 
with open("X_training_shakespeare_final.h5", "rb") as fp: 
    X = pickle.load(fp)
with open("y_training_shakespeare_final.h5", "rb") as fp: 
    y = pickle.load(fp)

In [ ]:
mapping = load(open("mapping.pkl", "rb"))
vocab_size = len(mapping)

In [ ]:
# Load from latest version of model 5 to continue training
model = load_model("model7_2.h5")
print(model.summary())

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'], loss_weights=[1, 0.2])
model.fit([X, X], [y, y], epochs=5, batch_size=256)

In [ ]:
# Save the trained model
filename = "model7_3.h5"
model.save(filename)

In [ ]:
# Download from colab to local disk
from google.colab import files
files.download(filename)